In [1]:
from matplotlib import pyplot as plt
import numpy as np

In [2]:
class Individual:
    def __init__(self, functions, lb, ub):
        self.functions = functions
        self.lb = lb
        self.ub = ub
        self.code = np.random.random(len(lb)) * (ub - lb) + lb
        self.fvalue = np.empty(len(functions))
        self.evaluate()

    def evaluate(self):
        for i in range(len(self.functions)):
            self.fvalue[i] = self.functions[i](self.code)

In [3]:
# Tournament selection

def selection(population, tournament_size):
    pass

In [4]:
# Whole arithmetic recombination

def crossover(parent1, parent2, child1, child2, alpha = 0.5):
    for i in range(len(child1.code)):
        child1.code[i] = parent1.code[i] * alpha + parent2.code[i] * (1 - alpha)
        child2.code[i] = parent2.code[i] * alpha + parent1.code[i] * (1 - alpha)

In [5]:
# Uniform mutation

def mutation(individual, mutation_prob):
    for i in range(len(individual.code)):
        if np.random.random() < mutation_prob:
            individual.code[i] = np.random.random() * (individual.ub[i] - individual.lb[i]) + individual.lb[i]

In [6]:
def ga(functions, lb, ub, pop_size, num_generations, tournament_size, elitism_size, mutation_prob):
    pass

In [7]:
# Bihn and Korn function without constraints
# 0 <= x <= 5
# 0 <= y <= 3

def f1(x):
    return 4*(x[0]**2 + x[1]**2)

def f2(x):
    return (x[0]-5)**2 + (x[1]-5)**2

lower_bounds = np.array([0, 0])
upper_bounds = np.array([5, 3])
varNum = 1000

i1 = Individual([f1, f2], lower_bounds, upper_bounds)
i2 = Individual([f1, f2], lower_bounds, upper_bounds)
print(i1.code, i1.fvalue)
print(i2.code, i2.fvalue)
child1 = Individual([f1, f2], lower_bounds, upper_bounds)
child2 = Individual([f1, f2], lower_bounds, upper_bounds)
crossover(i1, i2, child1, child2)
print(child1.code, child1.fvalue)
print(child2.code, child2.fvalue)
mutation(child1, 0.5)
print(child1.code)

[0.4476499  0.15809734] [ 0.9015408  44.16791284]
[1.15566483 2.34385693] [27.31690599 21.83400891]
[0.80165737 1.25097713] [14.63230835 30.55393552]
[0.80165737 1.25097713] [44.31096384 23.13021307]
[1.64325527 1.25097713]


In [8]:
# Viennet function
# -3 <= x,y <= 3

def f1(x):
    return 0.5*(x[0]**2 + x[1]**2) + np.sin(x[0]**2 + x[1]**2)

def f2(x):
    return ((3*x[0] - 2*x[1] + 4)**2)/8 + ((x[0] - x[1] + 1)**2)/27 + 15

def f3(x):
    return 1/(x[0]**2 + x[1]**2 + 1) - 1.1*np.exp(-(x[0]**2+x[1]**2))

lower_bounds = np.array([-3, -3])
upper_bounds = np.array([3, 3])
varNum = 1000

i1 = Individual([f1, f2, f3], lower_bounds, upper_bounds)
i2 = Individual([f1, f2, f3], lower_bounds, upper_bounds)
print(i1.code, i1.fvalue)
print(i2.code, i2.fvalue)
child1 = Individual([f1, f2, f3], lower_bounds, upper_bounds)
child2 = Individual([f1, f2, f3], lower_bounds, upper_bounds)
crossover(i1, i2, child1, child2)
print(child1.code, child1.fvalue)
print(child2.code, child2.fvalue)
mutation(child1, 0.5)
print(child1.code)

[-1.68597246 -1.63343263] [ 2.05731748 15.64317878  0.14914753]
[1.63696586 0.09748154] [ 1.78173476 24.73479046  0.19633263]
[-0.0245033  -0.76797555] [ 4.69047738 15.31504445  0.08133281]
[-0.0245033  -0.76797555] [ 1.79988716 17.02402241  0.15357554]
[-0.0245033  -0.70153385]
